In [1]:
import numpy as np
import pandas as pd
import os.path

%matplotlib inline
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, r'./../Python')
from helper import *

In [2]:
# This cell is solely for styling
import seaborn as sns
plt.style.use('seaborn')
plt.rcParams['grid.linestyle'] = "-"

The setup function ensures that everything in the input folder can be used. If pulled from Github, the input folder will only have zip files. This ensures that there are actual files in there that can be used as soon as the function is called.

The function also prints out each of the folder names that are in the Input folder.

In [3]:
setup()

../Input/AudioFiles
../Input/Docker
../Input/DockerV2
../Input/misc
../Input/NoProgram
../Input/Program


In [4]:
files = return_files('../Input/NoProgram')
files

['../Input/NoProgram/noProg.txt',
 '../Input/NoProgram/noProg2.txt',
 '../Input/NoProgram/noProg3.txt',
 '../Input/NoProgram/noProg4.txt',
 '../Input/NoProgram/noProg5.txt',
 '../Input/NoProgram/noProg6.txt',
 '../Input/NoProgram/noProg7.txt',
 '../Input/NoProgram/noProg8.txt']

In [5]:
info_dicts = []
for file in files:
    info_dicts.append(parse(file))

In [7]:
info_dicts

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
